In [38]:
import networkx as nx
import numpy as np
from pathlib import Path
import copy
import pandas as pd

from BNS_JT import trans, branch, variable, cpm, gen_bnb

HOME = Path.cwd()

In [39]:
# Network
node_coords = {'n1': (0, 0),
               'n2': (1, 1),
               'n3': (1, -1),
               'n4': (2, 0)}

arcs = {'e1': ['n1', 'n2'],
	'e2': ['n1', 'n3'],
	'e3': ['n2', 'n3'],
	'e4': ['n2', 'n4'],
	'e5': ['n3', 'n4']}
n_arc = len(arcs)

probs = {'e1': {0: 0.01, 1:0.99}, 'e2': {0:0.02, 1:0.98}, 'e3': {0:0.03, 1:0.97}, 'e4': {0:0.04, 1:0.96}, 'e5': {0:0.05, 1:0.95}}

od_pair=('n1','n4')

varis = {}
for k, v in arcs.items():
    varis[k] = variable.Variable( name=k, values = ['fail', 'surv'])

def conn(comps_st, od_pair, arcs): # connectivity analysis
    G = nx.Graph()
    for k,x in comps_st.items():
        if x > 0:
            G.add_edge(arcs[k][0], arcs[k][1], capacity=1)


    if od_pair[0] in G.nodes and od_pair[1] in G.nodes:
        f_val, _ = nx.maximum_flow(G,od_pair[0],od_pair[1])
    else:
        f_val = 0

    if f_val > 0:
        sys_st = 's'

        p = nx.shortest_path( G, od_pair[0], od_pair[1] )

        min_comps_st = {}
        for i in range(len(p)-1):
            pair = [p[i], p[i+1]]
            if pair in arcs.values():
                a = list(arcs.keys())[list(arcs.values()).index(pair)]
            else:
                a = list(arcs.keys())[list(arcs.values()).index([pair[1], pair[0]])]
            min_comps_st[a] = 1

    else:
        sys_st = 'f'
        min_comps_st = None

    return f_val, sys_st, min_comps_st

sys_fun = lambda comps_st : conn(comps_st, od_pair, arcs)

### Function 1. Give a rule decide a bound to run system function on.

* Depth-first decomposition

In [55]:
def decomp_df(varis, rules, probs, max_nb): # depth-first decomposition of event space using given rules

    worst = {x: 0 for x in varis.keys()}
    best = {k: len(v.values) - 1 for k, v in varis.items()}

    brs = gen_bnb.init_branch(worst, best, rules)
    brs_cr = [gen_bnb.get_compat_rules(brs[0].down, brs[0].up, rules)]
    brs_new = []
    brs_new_cr = []

    stop = False
    while stop == False:
        brs_crs = sorted(zip(brs, brs_cr), key=lambda x: x[0].p, reverse=True)
        brs, brs_cr = zip(*brs_crs)
        brs, brs_cr = list(brs), list(brs_cr)
        
        for i, (br, c_rules) in enumerate(zip(brs, brs_cr)):
            if (br.down_state == br.up_state) and (br.down_state != 'u'):
                brs_new.append(br)
                brs_new_cr.append({'s':[], 'f':[]})

            elif len(c_rules['f']) + len(c_rules['s']) < 1: # c_rules is empty                    
                brs_new.append(br)
                brs_new_cr.append({'s':[], 'f':[]})
            
            else:
                xd, xd_st = gen_bnb.get_decomp_comp_using_probs(br.down, br.up, c_rules, probs)
                
                # for upper
                up = br.up.copy()
                up[xd] = xd_st - 1
                up_state = gen_bnb.get_state(up, rules)
                p = gen_bnb.approx_branch_prob(br.down, up, probs)
                br_new = branch.Branch(br.down, up, br.down_state, up_state, p)
                brs_new.append(br_new)
                if (br_new.down_state == br_new.up_state) and (br_new.up_state != 'u'):
                    brs_new_cr.append({'s':[], 'f':[]})
                else:
                    cr_new = gen_bnb.get_compat_rules(br_new.down, br_new.up, rules)
                    brs_new_cr.append(cr_new)

                # for lower
                down = br.down.copy()
                down[xd] = xd_st
                down_state = gen_bnb.get_state(down, rules)
                p = gen_bnb.approx_branch_prob(down, br.up, probs)
                br_new = branch.Branch(down, br.up, down_state, br.up_state, p)
                brs_new.append(br_new)
                if (br_new.down_state == br_new.up_state) and (br_new.down_state != 'u'):
                    brs_new_cr.append({'s':[], 'f':[]})
                else:
                    cr_new = gen_bnb.get_compat_rules(br_new.down, br_new.up, rules)
                    brs_new_cr.append(cr_new)

                n_br = len(brs_new) + (len(brs)-(i+1)) # the current number of branches
                if n_br >= max_nb:
                    stop = True

                    try:
                        brs_new = brs_new + brs[(i+1):]
                        brs_new_cr = brs_new_cr + brs_cr[(i+1):]
                    except TypeError:
                        ddd = 1
                    break

        brs = copy.deepcopy( brs_new )
        brs_cr = copy.deepcopy( brs_new_cr )
        brs_new = []
        brs_new_cr = []

        if stop==False:
            ncr = [len(cr['f'])+len(cr['s']) for cr in brs_cr]
            if all(x==0 for x in ncr):
                stop = True

    return brs, brs_cr

In [41]:
# Example
ex1 = {'rules': {'s': [{'e1': 1, 'e4': 1}, {'e2': 1, 'e5': 1}], 'f': []}}
ex2 = {'rules': {'s': [{'e1': 1, 'e4': 1}, {'e2': 1, 'e5': 1}], 'f': [{'e4': 0, 'e5':0}]}}

In [42]:
ex1['brs'], ex1['brs_cr'] = decomp_df(varis, ex1['rules'], probs, max_nb=10)
print('Example 1 ..')
for b in ex1['brs']:
    print(b)
kk = sum(b.p for b in ex1['brs'])
print(f'Prob sum: {kk} \n')

ex2['brs'], ex2['brs_cr'] = decomp_df(varis, ex2['rules'], probs, max_nb=10)
print('Example 2 ..')
for b in ex2['brs']:
    print(b)
kk = sum(b.p for b in ex2['brs'])
print(f'Prob sum: {kk}')

Example 1 ..
Branch(down={'e1': 1, 'e2': 0, 'e3': 0, 'e4': 1, 'e5': 0}, up={'e1': 1, 'e2': 1, 'e3': 1, 'e4': 1, 'e5': 1}, down_state='s', up_state='s', p=0.9503999999999999)
Branch(down={'e1': 1, 'e2': 1, 'e3': 0, 'e4': 0, 'e5': 0}, up={'e1': 1, 'e2': 1, 'e3': 1, 'e4': 0, 'e5': 0}, down_state='u', up_state='u', p=0.0019404000000000001)
Branch(down={'e1': 1, 'e2': 1, 'e3': 0, 'e4': 0, 'e5': 1}, up={'e1': 1, 'e2': 1, 'e3': 1, 'e4': 0, 'e5': 1}, down_state='s', up_state='s', p=0.0368676)
Branch(down={'e1': 0, 'e2': 1, 'e3': 0, 'e4': 0, 'e5': 1}, up={'e1': 0, 'e2': 1, 'e3': 1, 'e4': 1, 'e5': 1}, down_state='s', up_state='s', p=0.009309999999999999)
Branch(down={'e1': 1, 'e2': 0, 'e3': 0, 'e4': 0, 'e5': 0}, up={'e1': 1, 'e2': 0, 'e3': 1, 'e4': 0, 'e5': 1}, down_state='u', up_state='u', p=0.0007920000000000001)
Branch(down={'e1': 0, 'e2': 1, 'e3': 0, 'e4': 0, 'e5': 0}, up={'e1': 0, 'e2': 1, 'e3': 1, 'e4': 1, 'e5': 0}, down_state='u', up_state='u', p=0.00049)
Branch(down={'e1': 0, 'e2': 0, 'e

### Function 2: Pick the state for system analysis

In [43]:
def get_st_decomp( brs ): # 'brs' is a list of branches obtained by depth-first decomposition

    brs = sorted(brs, key=lambda x: x.p, reverse=True)
    x_star = None
    for b in brs: # look at up_state first
        if b.up_state == 'u':
            x_star = b.up
            break

    if x_star == None:
        for b in brs: # if all up states are known then down states
            if b.down_state == 'u':
                x_star = b.down
                break

    return x_star

In [44]:
ex1['x_star'] = get_st_decomp(ex1['brs'])
print(ex1['x_star'])

ex2['x_star'] = get_st_decomp(ex2['brs'])
print(ex2['x_star'])

{'e1': 1, 'e2': 1, 'e3': 1, 'e4': 0, 'e5': 0}
{'e1': 1, 'e2': 0, 'e3': 1, 'e4': 0, 'e5': 1}


### Final decomposition algorithm: Now put them all together.

In [45]:
max_nb = 100
max_sf = 100

In [56]:
def run_brc(varis, probs, max_sf, max_nb):

    rules = {'s': [], 'f': []}
    sys_res = pd.DataFrame(data={'sys_val': [], 'comp_st': [], 'comp_st_min': []}) # system function results

    monitor = {'pf_up': [], # upper bound on pf
               'pf_low': [], # lower bound on pf
               'br_ns': [], # number of branches
               'r_ns': [], # number of rules
               'sf_ns': [] # number of system function runs
              }

    no_sf = 0
    while no_sf < max_sf:
        brs, _ = decomp_df(varis, rules, probs, max_nb)
        x_star = get_st_decomp( brs )

        if x_star == None or len(brs) >= max_nb:
            break # all braches have been specified or number of branches reached its max.
        else:
            rule, sys_res_ = gen_bnb.run_sys_fn(x_star, sys_fun, varis)

            rules = gen_bnb.update_rule_set(rules, rule)
            sys_res = pd.concat([sys_res, sys_res_], ignore_index=True)
            no_sf += 1

            ######## monitoring ############
            pr_bf = sum([b[4] for b in brs if b.up_state == 'f']) # prob. of failure branches
            pr_bs = sum([b[4] for b in brs if b.down_state == 's']) # prob. of survival branches
        
            no_rf = len(rules['f'])
            no_rs = len(rules['s'])

            monitor['r_ns'].append( no_rf + no_rs )
            monitor['pf_up'].append(1.0-pr_bs)
            monitor['pf_low'].append(pr_bf)
            monitor['br_ns'].append(len(brs))
            monitor['sf_ns'].append(no_sf)

            no_bf = sum([b.up_state == 'f' for b in brs])
            no_bs = sum([b.down_state == 's' for b in brs])
            no_bu = len(brs) - no_bf - no_bs

            try:
                min_len_rf = min( [len(x) for x in rules['f']] )
            except ValueError:
                min_len_rf = 0


            print(f'[System function runs {no_sf}]..')
            print(f'The # of found non-dominated rules (f, s): {no_rf + no_rs} ({no_rf}, {no_rs})')
            print(f'Probability of branchs (f, s, u): ({pr_bf:.2f}, {pr_bs:.2f}, {1.0-pr_bs-pr_bf:.2f})')
            print(f'The # of branches (f, s, u), min len of rf: {len(brs)} ({no_bf}, {no_bs}, {no_bu}), {min_len_rf}')
            #################################


    brs, _ = decomp_df(varis, rules, probs, max_nb)

    return brs, rules, sys_res, monitor

In [47]:
brs, rules, sys_res, monitor = run_brc(varis, probs, max_sf, max_nb)
print(monitor['sf_ns'][-1])
print(brs)
print(rules)
print(monitor['r_ns'])
print(monitor['pf_up'])
print(monitor['pf_low'])

[System function runs 1]..
The # of found non-dominated rules (f, s): 1 (0, 1)
Probability of branchs (f, s, u): (0.00, 0.00, 1.00)
The # of branches (f, s, u), min len of rf: 1 (0, 0, 1), 0
[System function runs 2]..
The # of found non-dominated rules (f, s): 2 (0, 2)
Probability of branchs (f, s, u): (0.00, 0.95, 0.05)
The # of branches (f, s, u), min len of rf: 3 (0, 1, 2), 0
[System function runs 3]..
The # of found non-dominated rules (f, s): 3 (1, 2)
Probability of branchs (f, s, u): (0.00, 1.00, 0.00)
The # of branches (f, s, u), min len of rf: 7 (0, 3, 4), 2
[System function runs 4]..
The # of found non-dominated rules (f, s): 4 (1, 3)
Probability of branchs (f, s, u): (0.00, 1.00, 0.00)
The # of branches (f, s, u), min len of rf: 7 (1, 3, 3), 2
[System function runs 5]..
The # of found non-dominated rules (f, s): 5 (1, 4)
Probability of branchs (f, s, u): (0.00, 1.00, 0.00)
The # of branches (f, s, u), min len of rf: 9 (2, 4, 3), 2
[System function runs 6]..
The # of found non

## Now to EMA

In [48]:
import random
from BNS_JT import config

random.seed(1)

Randomly divide edges into three groups with different failure probabilities for demonstration's sake.

In [49]:
cfg = config.Config('C:/Users/jb622s/git/BNS-JT/BNS_JT/demos/ema/config.json')
st_br_to_cs = {'f':0, 's':1, 'u': 2}

n_edge = len(cfg.infra['edges'])

integers = list(range(n_edge))
random.shuffle(integers)
group_size = len(integers) // 3

prob_groups = [sorted(integers[:group_size]), sorted(integers[group_size:2*group_size]), sorted(integers[2*group_size:])]
probs_setting = [{0:0.01, 1: 0.04, 2: 0.95}, {0:0.03, 1: 0.12, 2: 0.85}, {0:0.06, 1: 0.24, 2: 0.70}]

probs = {}
for i in range(n_edge):
    g_idx = next(index for index, group in enumerate(prob_groups) if i in group)
    probs['e'+str(i+1)] = probs_setting[g_idx]

print(probs)

{'e1': {0: 0.06, 1: 0.24, 2: 0.7}, 'e2': {0: 0.06, 1: 0.24, 2: 0.7}, 'e3': {0: 0.06, 1: 0.24, 2: 0.7}, 'e4': {0: 0.06, 1: 0.24, 2: 0.7}, 'e5': {0: 0.01, 1: 0.04, 2: 0.95}, 'e6': {0: 0.01, 1: 0.04, 2: 0.95}, 'e7': {0: 0.01, 1: 0.04, 2: 0.95}, 'e8': {0: 0.01, 1: 0.04, 2: 0.95}, 'e9': {0: 0.01, 1: 0.04, 2: 0.95}, 'e10': {0: 0.01, 1: 0.04, 2: 0.95}, 'e11': {0: 0.01, 1: 0.04, 2: 0.95}, 'e12': {0: 0.03, 1: 0.12, 2: 0.85}, 'e13': {0: 0.06, 1: 0.24, 2: 0.7}, 'e14': {0: 0.06, 1: 0.24, 2: 0.7}, 'e15': {0: 0.01, 1: 0.04, 2: 0.95}, 'e16': {0: 0.06, 1: 0.24, 2: 0.7}, 'e17': {0: 0.06, 1: 0.24, 2: 0.7}, 'e18': {0: 0.01, 1: 0.04, 2: 0.95}, 'e19': {0: 0.03, 1: 0.12, 2: 0.85}, 'e20': {0: 0.01, 1: 0.04, 2: 0.95}, 'e21': {0: 0.01, 1: 0.04, 2: 0.95}, 'e22': {0: 0.01, 1: 0.04, 2: 0.95}, 'e23': {0: 0.01, 1: 0.04, 2: 0.95}, 'e24': {0: 0.03, 1: 0.12, 2: 0.85}, 'e25': {0: 0.03, 1: 0.12, 2: 0.85}, 'e26': {0: 0.03, 1: 0.12, 2: 0.85}, 'e27': {0: 0.06, 1: 0.24, 2: 0.7}, 'e28': {0: 0.06, 1: 0.24, 2: 0.7}, 'e29': {0:

In [50]:
varis = {}
cpms = {}
for k in cfg.infra['edges'].keys():
    varis[k] = variable.Variable(name=k, values = cfg.scenarios['scenarios']['s1'][k])
    cpms[k] = cpm.Cpm(variables = [varis[k]], no_child=1,
                        C = np.array([0, 1, 2]).T, p = [probs[k][0], probs[k][1], probs[k][2]])


sys_fun = trans.sys_fun_wrap(od_pair, cfg.infra['edges'], varis)

thres = 2
st_br_to_cs = {'f': 0, 's': 1, 'u': 2}

#csys_by_od, varis_by_od = {}, {}
# branches by od_pair
#for k, od_pair in cfg.infra['ODs'].items():
comps_st_itc = {k: len(v.values) - 1 for k, v in varis.items()}
#od_pair = ('n1', 'n14')
od_pair = ('n1','n53')
d_time_itc, path_itc = trans.get_time_and_path_given_comps(comps_st_itc, od_pair, cfg.infra['edges'], varis)

# system function
sys_fun = trans.sys_fun_wrap(od_pair, cfg.infra['edges'], varis, thres * d_time_itc)

In [57]:
brs, rules, sys_res, monitor = run_brc(varis, probs, max_sf=1000, max_nb=1000)

[System function runs 1]..
The # of found non-dominated rules (f, s): 1 (0, 1)
Probability of branchs (f, s, u): (0.00, 0.00, 1.00)
The # of branches (f, s, u), min len of rf: 1 (0, 0, 1), 0
[System function runs 2]..
The # of found non-dominated rules (f, s): 2 (0, 2)
Probability of branchs (f, s, u): (0.00, 0.20, 0.80)
The # of branches (f, s, u), min len of rf: 8 (0, 1, 7), 0
[System function runs 3]..
The # of found non-dominated rules (f, s): 3 (0, 3)
Probability of branchs (f, s, u): (0.00, 0.29, 0.71)
The # of branches (f, s, u), min len of rf: 12 (0, 3, 9), 0
[System function runs 4]..
The # of found non-dominated rules (f, s): 4 (0, 4)
Probability of branchs (f, s, u): (0.00, 0.35, 0.65)
The # of branches (f, s, u), min len of rf: 16 (0, 4, 12), 0
[System function runs 5]..
The # of found non-dominated rules (f, s): 4 (0, 4)
Probability of branchs (f, s, u): (0.00, 0.39, 0.61)
The # of branches (f, s, u), min len of rf: 46 (0, 7, 39), 0
[System function runs 6]..
The # of foun

In [58]:
print(monitor['sf_ns'][-1])
print(len(brs))
print(rules)
print(monitor['r_ns'])
print(monitor['pf_up'])
print(monitor['pf_low'])

30
1000
{'s': [{'e3': 2, 'e19': 2, 'e24': 1, 'e27': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e2': 2, 'e15': 2, 'e24': 2, 'e26': 2, 'e31': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e3': 2, 'e19': 2, 'e24': 2, 'e26': 2, 'e31': 2, 'e42': 2, 'e56': 2, 'e75': 2, 'e84': 2, 'e89': 2, 'e92': 2, 'e107': 2}, {'e2': 2, 'e15': 2, 'e24': 1, 'e27': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e3': 2, 'e19': 2, 'e24': 2, 'e27': 2, 'e43': 2, 'e57': 2, 'e80': 2, 'e81': 2, 'e98': 2}, {'e1': 2, 'e5': 2, 'e13': 2, 'e31': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e3': 2, 'e19': 2, 'e24': 1, 'e27': 2, 'e42': 2, 'e56': 2, 'e75': 2, 'e84': 2, 'e89': 2, 'e92': 2, 'e107': 2}, {'e3': 2, 'e19': 2, 'e24': 2, 'e26': 2, 'e28': 2, 'e30': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e2': 2, 'e15': 2, 'e24': 2, 'e26': 2, 'e31': 2, 'e42': 2, 'e56': 2, 'e75': 2, 'e84': 2, 'e89': 2, 'e92': 2, 'e107': 2}, {'e3': 2, 'e19': 2, 'e24': 2, 'e27': 2, 'e44': 2, 'e78': 2, 'e79': 2, 'e98': 2}, {'e1': 2, 'e5': 2, 'e13': 2, 'e31': 2, 'e42': 2, 'e56': 2, 'e75': 2, 'e84